# Pre-eminence tutorial

In this tutorial, you'll learn how to define an agent to play the game Pre-eminence, run games so you can track performance, and step through those games so you can debug & understand what your agent is doing at each point.

## 1. Creating an agent

In [ ]:
%load_ext autoreload
%autoreload 2
import preem as P
import random

In [ ]:
class RandomAgent(P.Agent):
    def place(self, state):
        return random.choice(state.my_territories)

    def reinforce(self, state, count):
        return {random.choice(state.my_territories): count}

    def redeem(self, state):
        if 5 <= len(state.cards):
            sets = list(P.get_matching_sets(state.cards))
            return random.choice(sets)

    def act(self, state, earned_card):
        possible_attacks = [a for a in P.get_all_possible_attacks(state) if a.count >= 2]
        if possible_attacks:
            return random.choices(possible_attacks, weights=[a.count for a in possible_attacks])[0]
        possible_moves = P.get_all_possible_moves(state)
        if possible_moves:
            return random.choices(possible_moves, weights=[a.count for a in possible_moves])[0]
        return None

In [ ]:
random.seed(10)
P.Game.play(P.Map.load_file('maps/mini.json'), [RandomAgent(), RandomAgent()])

In [ ]:
random.seed(10)
for event in P.Game.start(P.Map.load_file('maps/mini.json'), [RandomAgent(), RandomAgent()]):
    print(id(event.agent), event.method, event.args, '->', event.result)

## 2. Looking at the world

In [ ]:
random.seed(10)
game = P.Game.start(P.Map.load_file('maps/mini.json'), [RandomAgent(), RandomAgent()])
print(game.world)
game.map  # same as game.world.map

In [ ]:
event = next(game)
print(event)
print(event.state)
print(event.state.world)
print('territory owners', event.state.world.owners)
event

In [ ]:
while event.method != 'act':
    event = next(game)
print(event.result)
event

## 3. Running a mini-tournament

In [ ]:
# TODO